## General

This section of for imports, getting an overview of all experiments, and loading the most suitable model.

In [4]:
import sys
import os
import warnings
import logging
from absl import logging as absl_logging

# ✅ Ensure Correct TensorFlow Configuration
os.environ['TFDS_DATA_DIR'] = r"/data/newc6477/VAE/Single_Beat/5_percent_Physionet/"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

# ✅ Ensure Correct Working Directory
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"📂 Project Root: {PROJECT_ROOT}")

# ✅ Ensure `src/` is in Python's Path
SRC_DIR = os.path.join(PROJECT_ROOT, "src")
if SRC_DIR not in sys.path:
    sys.path.append(SRC_DIR)

print(f"✅ Updated sys.path:\n{sys.path}")

# ✅ Suppress Warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)
absl_logging.set_verbosity(absl_logging.ERROR)

# ✅ Import Modules
try:
    from src.utils.helper import Helper
    from src.evaluate.visualizations import Visualizations
    print("✅ Successfully imported `Helper` and `Visualizations`")
except ModuleNotFoundError as e:
    print(f"❌ Import Error: {e}")
    print("🔍 Check if `src/` has `__init__.py` and is in `sys.path`.")

# ✅ Import Other Required Libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
from neurokit2.signal import signal_smooth

from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split

import ipywidgets as widgets
from matplotlib import pyplot as plt
import seaborn as sns



📂 Project Root: /users/newc6477/VAE/12_Lead_VECG
✅ Updated sys.path:
['/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python310.zip', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/lib-dynload', '', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/site-packages', '/users/newc6477/Benchmark_ISIBrno/ENTER/envs/my_env/lib/python3.10/site-packages/setuptools/_vendor', '/users/newc6477/VAE/12_Lead_VECG', '/users/newc6477/VAE/12_Lead_VECG/src']


✅ Successfully imported `Helper` and `Visualizations`


In [5]:
# The resolution for saving images
DPI = 300

# The source path of the experiments and models
PATH = r"/users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1"

# Some operations take some time in computation.
# Therefore, the stored intermediate results can be used to skip the respective computation.
USE_PRECOMPUTED = True

# 2. Anomaly Detection

## 🔹 Funcs used to train and test

In [6]:
import json
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
import numpy as np
import pandas as pd

def evaluate_models(df_physionet_train, df_physionet_test, hyperparams_list):
    """
    Evaluate a list of models using KNN classification and return a summary table.
    
    Args:
        df_physionet_train (List[pd.DataFrame]): List of train embedding DataFrames (1 per model).
        df_physionet_test (List[pd.DataFrame]): List of test embedding DataFrames (1 per model).
        hyperparams_list (List[dict]): Corresponding list of hyperparameter dictionaries.
    
    Returns:
        pd.DataFrame: Summary table with accuracy, F1 scores, hyperparameters.
    """
    results = []

    for i in range(len(df_physionet_train)):
        print(f"\n🔍 Evaluating model {i+1}/{len(df_physionet_train)}...")

        # Extract hyperparameters and latent dimension
        hparams = hyperparams_list[i]
        latent_dim = hparams.get("latent_dimension", 8)

        # Prepare feature matrices and labels
        X_train = df_physionet_train[i].iloc[:, :latent_dim].values  
        X_test = df_physionet_test[i].iloc[:, :latent_dim].values  
        y_train = np.array(df_physionet_train[i]['diagnostic'].tolist(), dtype=int)  
        y_test = np.array(df_physionet_test[i]['diagnostic'].tolist(), dtype=int)  

        # KNN classification with hyperparameter tuning
        knn = KNeighborsClassifier()
        multi_knn = MultiOutputClassifier(knn, n_jobs=-1)

        param_grid = {"estimator__n_neighbors": [3, 5, 7, 9, 11]}
        grid = GridSearchCV(multi_knn, param_grid, scoring="accuracy", cv=3, n_jobs=-1)
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)

        # Evaluation metrics
        overall_acc = accuracy_score(y_test, y_pred)
        f1_micro = f1_score(y_test, y_pred, average="micro")
        f1_macro = f1_score(y_test, y_pred, average="macro")
        accuracy_per_label = (y_pred == y_test).mean(axis=0)

        # Print evaluation
        print("✅ Overall Accuracy:", overall_acc)
        print("✅ F1 Score (Macro):", f1_macro)
        print("✅ F1 Score (Micro):", f1_micro)
        print("✅ Accuracy per label:", accuracy_per_label)

        # Classification report
        print("✅ Classification Report:\n")
        print(classification_report(
            y_test, y_pred,
            target_names=[str(i) for i in range(y_test.shape[1])]
        ))

        # Collect results
        results.append({
            "model_index": i,
            "best_k": grid.best_params_["estimator__n_neighbors"],
            "accuracy": overall_acc,
            "f1_micro": f1_micro,
            "f1_macro": f1_macro,
            **hparams
        })

    summary_df = pd.DataFrame(results)
    return summary_df


def extract_hyperparams_from_json(json_path):
    with open(json_path, 'r') as f:
        params = json.load(f)

    # Extract key hyperparameters
    hyperparams = {
        "alpha": params["coefficients"]["alpha"],
        "beta": params["coefficients"]["beta"],
        "gamma": params["coefficients"]["gamma"],
        "latent_dimension": params["latent_dimension"],
        "learning_rate": params["learning_rate"],
        "epochs": params["epochs"]
    }

    return hyperparams

from IPython.display import display
def summarize_by_latent_dimension(results_df):
    unique_dims = results_df["latent_dimension"].unique()
    summary_tables = {}

    for dim in sorted(unique_dims):
        filtered = results_df[results_df["latent_dimension"] == dim]
        sorted_table = filtered.sort_values(by="f1_macro", ascending=False)
        summary_tables[dim] = sorted_table

        print(f"\n📏 Latent Dimension: {dim}")
        display(sorted_table)

    return summary_tables


## 🔹 Lead I Evaluation

In [13]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'I')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/I/2025-03-28_15-47-53/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/I/2025-03-28_15-56-40/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/I/2025-03-28_16-05-14/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/I/2025-03-28_16-13-50/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/I/2025-03-28_16-22-58/model_best.keras
Loaded 5 models successfully.


In [14]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}

In [15]:
df_physionet_train, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='I')
df_physionet_test, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='I')

#df_physionet_train = df_physionet_train[0]
#df_physionet_test = df_physionet_test[0]
#print(type(df_physionet_train))



📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: I
     42/Unknown - 3s 57ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 57ms/step
     42/Unknown - 3s 62ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 61ms/step
     43/Unknown - 3s 61ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 60ms/step
     42/Unknown - 3s 61ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 61ms/step
     42/Unknown - 3s 62ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 61ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: I
      9/Unknown - 0s 53ms/step✅ Generator exhausted normally.
11/11 [==============================] - 1s 51ms/step
      9/Unknown - 0s 52ms/step✅ Generator exhausted normally.
11/11 [==============================] - 1s 49ms/step
      9/Unknown - 0s 51ms/step✅ Generator exhausted normally.
11/11 [======================

In [16]:

results_df = evaluate_models(df_physionet_train,df_physionet_test, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.29830960854092525
✅ F1 Score (Macro): 0.15611490698945577
✅ F1 Score (Micro): 0.3171412124164194
✅ Accuracy per label: [0.91307829 0.88202847 0.97544484 0.99741993 0.98781139 0.96058719
 0.94928826 0.9774911  0.92135231 0.97411032 0.99626335 0.97927046
 0.97482206 0.98629893 0.73158363 0.94510676 0.99510676 0.9905694
 0.98247331 0.97259786 0.97508897 0.95685053 0.89145907 0.88033808
 0.86690391 0.96467972]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.21      0.08      0.12       791
           1       0.38      0.23      0.28      1155
           2       0.17      0.00      0.01       272
           3       0.00      0.00      0.00         6
           4       0.39      0.19      0.25       124
           5       0.69      0.43      0.53       580
           6       0.07      0.06      0.06       319
           7       0.08      0.03      0.05       186
           8       0.16      0.04      0.07       740
 

In [17]:
display(results_df.sort_values(by="f1_macro", ascending=False))


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,5,0.298310,0.317141,0.156115,2.01,0.3,0.2,20,0.002,100
1,1,5,0.283274,0.314268,0.148389,2.01,0.3,0.2,24,0.002,100
4,4,9,0.298488,0.319012,0.142426,2.01,0.3,0.2,22,0.002,100
2,2,11,0.311032,0.313001,0.141885,2.01,0.3,0.2,16,0.002,100
3,3,7,0.281584,0.304471,0.140049,2.01,0.3,0.2,18,0.002,100


## 🔹 Lead II Evaluation


In [7]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'II')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/II/2025-03-28_16-31-34/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/II/2025-03-28_16-40-14/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/II/2025-03-28_16-48-36/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/II/2025-03-28_16-57-28/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/II/2025-03-28_17-06-06/model_best.keras
Loaded 5 models successfully.


In [8]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_II, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='II')
df_physionet_test_II, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='II')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: II
     43/Unknown - 3s 64ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 64ms/step
     42/Unknown - 3s 65ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 64ms/step
     42/Unknown - 3s 63ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 62ms/step
     42/Unknown - 3s 63ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 62ms/step
     42/Unknown - 3s 60ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 60ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: II
     10/Unknown - 1s 53ms/step✅ Generator exhausted normally.
11/11 [==============================] - 1s 51ms/step
     10/Unknown - 1s 52ms/step✅ Generator exhausted normally.
11/11 [==============================] - 1s 50ms/step
      9/Unknown - 0s 52ms/step✅ Generator exhausted normally.
11/11 [====================

In [9]:
results_df_II = evaluate_models(df_physionet_train_II,df_physionet_test_II, hyperparams_list)


🔍 Evaluating model 1/5...


✅ Overall Accuracy: 0.25195729537366546
✅ F1 Score (Macro): 0.1309770027123678
✅ F1 Score (Micro): 0.30497237569060776
✅ Accuracy per label: [0.91325623 0.87117438 0.9747331  0.99848754 0.98745552 0.93576512
 0.96441281 0.97624555 0.91903915 0.96841637 0.99635231 0.97633452
 0.97170819 0.9816726  0.7091637  0.94030249 0.98185053 0.99048043
 0.97935943 0.97144128 0.97571174 0.95106762 0.8819395  0.86619217
 0.84964413 0.95685053]
✅ Classification Report:

              precision    recall  f1-score   support

           0       0.31      0.19      0.23       791
           1       0.33      0.25      0.28      1155
           2       0.27      0.03      0.05       272
           3       0.00      0.00      0.00         6
           4       0.19      0.04      0.07       124
           5       0.25      0.13      0.17       580
           6       0.22      0.10      0.14       319
           7       0.01      0.01      0.01       186
           8       0.32      0.20      0.24       740


In [10]:
display(results_df_II.sort_values(by="f1_macro", ascending=False))
summarized_tables = summarize_by_latent_dimension(results_df_II)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,3,0.261477,0.324687,0.144117,2.01,0.3,0.2,22,0.002,100
1,1,3,0.252313,0.324931,0.136799,2.01,0.3,0.2,24,0.002,100
3,3,3,0.258363,0.320848,0.136355,2.01,0.3,0.2,18,0.002,100
0,0,3,0.251957,0.304972,0.130977,2.01,0.3,0.2,20,0.002,100
2,2,7,0.266370,0.312171,0.120777,2.01,0.3,0.2,16,0.002,100



📏 Latent Dimension: 16


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,7,0.26637,0.312171,0.120777,2.01,0.3,0.2,16,0.002,100



📏 Latent Dimension: 18


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,3,0.258363,0.320848,0.136355,2.01,0.3,0.2,18,0.002,100


...

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,3,0.251957,0.304972,0.130977,2.01,0.3,0.2,20,0.002,100



📏 Latent Dimension: 22


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,3,0.261477,0.324687,0.144117,2.01,0.3,0.2,22,0.002,100



📏 Latent Dimension: 24


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,3,0.252313,0.324931,0.136799,2.01,0.3,0.2,24,0.002,100


## 🔹 Lead III Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'III')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/III/2025-03-28_17-14-53/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/III/2025-03-28_17-23-11/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/III/2025-03-28_17-32-09/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/III/2025-03-28_17-40-49/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/III/2025-03-28_17-49-56/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_III, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='III')
df_physionet_test_III, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='III')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: III
     42/Unknown - 2s 39ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 39ms/step
     43/Unknown - 2s 35ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 35ms/step
     43/Unknown - 2s 35ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 35ms/step
     41/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     43/Unknown - 2s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: III
      8/Unknown - 0s 28ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 29ms/step
      9/Unknown - 0s 25ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 28ms/step
  

In [ ]:
results_df_III = evaluate_models(df_physionet_train_III,df_physionet_test_III, hyperparams_list)


🔍 Evaluating model 1/5...



🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_III.sort_values(by="f1_macro", ascending=False))
summarized_tables = summarize_by_latent_dimension(results_df_III)

# Loop through and display each
for latent_dim, table in summarized_tables.items():
    print(f"🔢 Latent Dimension: {latent_dim}")
    display(table)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,3,0.208452,0.267195,0.122479,2.01,0.3,0.2,18,0.002,100
0,0,3,0.212011,0.286967,0.119266,2.01,0.3,0.2,20,0.002,100
2,2,3,0.209431,0.276149,0.115711,2.01,0.3,0.2,16,0.002,100
1,1,3,0.191370,0.257746,0.110634,2.01,0.3,0.2,24,0.002,100
4,4,11,0.224911,0.277264,0.096398,2.01,0.3,0.2,22,0.002,100



📏 Latent Dimension: 16


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,3,0.209431,0.276149,0.115711,2.01,0.3,0.2,16,0.002,100



📏 Latent Dimension: 18


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,3,0.208452,0.267195,0.122479,2.01,0.3,0.2,18,0.002,100


...

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,3,0.212011,0.286967,0.119266,2.01,0.3,0.2,20,0.002,100


🔢 Latent Dimension: 22


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,11,0.224911,0.277264,0.096398,2.01,0.3,0.2,22,0.002,100


🔢 Latent Dimension: 24


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,3,0.19137,0.257746,0.110634,2.01,0.3,0.2,24,0.002,100


## 🔹 Lead aVR Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVR')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVR/2025-03-28_22-46-04/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVR/2025-03-28_22-54-26/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVR/2025-03-28_23-07-03/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVR/2025-03-28_23-19-13/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVR/2025-03-28_23-31-22/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVR, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVR')
df_physionet_test_aVR, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVR')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: aVR
     43/Unknown - 2s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     43/Unknown - 2s 36ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 36ms/step
     41/Unknown - 2s 36ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 35ms/step
     41/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step
     43/Unknown - 2s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVR
      9/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 26ms/step
      8/Unknown - 0s 27ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 26ms/step
     10/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==================

In [ ]:
results_df_aVR = evaluate_models(df_physionet_train_aVR,df_physionet_test_aVR, hyperparams_list)

NameError: name 'df_physionet_train_aVR' is not defined

In [ ]:
display(results_df_aVR.sort_values(by="f1_macro", ascending=False))
#summarized_tables = summarize_by_latent_dimension(results_df_aVR)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,3,0.288790,0.334247,0.157934,2.01,0.3,0.2,18,0.002,100
2,2,3,0.289769,0.346468,0.153058,2.01,0.3,0.2,16,0.002,100
4,4,7,0.302847,0.351322,0.151326,2.01,0.3,0.2,22,0.002,100
1,1,3,0.295374,0.342358,0.149792,2.01,0.3,0.2,24,0.002,100
0,0,3,0.285676,0.336031,0.149707,6.01,0.3,0.2,20,0.002,100


## 🔹 Lead aVL Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVL')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

2025-03-29 14:59:06.845284: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-29 14:59:07.063798: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-29 14:59:07.572631: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-29 14:59:07.572757: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-29 14:59:07.574591: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

NameError: name 'PATH' is not defined

In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVL, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVL')
df_physionet_test_aVL, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVL')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: aVL
     42/Unknown - 2s 38ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 38ms/step
     43/Unknown - 2s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     43/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     41/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step
     42/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVL
     10/Unknown - 0s 25ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
      8/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
  

In [ ]:
results_df_aVL = evaluate_models(df_physionet_train_aVL,df_physionet_test_aVL, hyperparams_list)


🔍 Evaluating model 1/5...



🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_aVL.sort_values(by="f1_macro", ascending=False))
#summarized_tables = summarize_by_latent_dimension(results_df_aVL)

NameError: name 'results_df_aVL' is not defined

## aVF

In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'aVF')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVF/2025-03-29_03-02-23/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVF/2025-03-29_03-10-40/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVF/2025-03-29_03-23-17/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVF/2025-03-29_03-35-37/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/aVF/2025-03-29_03-48-11/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_aVF, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='aVF')
df_physionet_test_aVF, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='aVF')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: aVF
     41/Unknown - 2s 36ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 36ms/step
     41/Unknown - 1s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     43/Unknown - 2s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     41/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step
     43/Unknown - 1s 31ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 31ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: aVF
      9/Unknown - 0s 24ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 25ms/step
      8/Unknown - 0s 28ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
      9/Unknown - 0s 24ms/step✅ Generator exhausted normally.
11/11 [==================

In [ ]:
results_df_aVF = evaluate_models(df_physionet_train_aVF,df_physionet_test_aVF, hyperparams_list)


🔍 Evaluating model 1/5...

🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_aVF.sort_values(by="f1_macro", ascending=False))
#summarized_tables = summarize_by_latent_dimension(results_df_V2)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,3,0.229893,0.298067,0.130458,2.01,0.3,0.2,18,0.002,100
4,4,3,0.222776,0.290578,0.128266,2.01,0.3,0.2,22,0.002,100
1,1,3,0.217972,0.286289,0.126114,2.01,0.3,0.2,24,0.002,100
0,0,3,0.228025,0.275270,0.121104,6.01,0.3,0.2,20,0.002,100
2,2,3,0.229626,0.280700,0.115907,2.01,0.3,0.2,16,0.002,100


## 🔹 Lead V1 Evaluation

In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V1')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V1/2025-03-29_04-00-38/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V1/2025-03-29_04-13-06/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V1/2025-03-29_04-25-29/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V1/2025-03-29_04-38-57/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V1/2025-03-29_04-52-24/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V1, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V1')
df_physionet_test_V1, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V1')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V1
     42/Unknown - 2s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     43/Unknown - 2s 35ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 35ms/step
     43/Unknown - 2s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     43/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     41/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V1
     10/Unknown - 0s 27ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 26ms/step
      8/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
    

In [ ]:
results_df_V1 = evaluate_models(df_physionet_train_V1,df_physionet_test_V1, hyperparams_list)


🔍 Evaluating model 1/5...



🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_V1.sort_values(by="f1_macro", ascending=False))
#summarized_tables = summarize_by_latent_dimension(results_df_aVL)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,3,0.225534,0.291844,0.135277,2.01,0.3,0.2,24,0.002,100
4,4,3,0.226157,0.294748,0.129856,2.01,0.3,0.2,22,0.002,100
0,0,3,0.225623,0.295866,0.129098,6.01,0.3,0.2,20,0.002,100
2,2,3,0.236388,0.290082,0.128045,2.01,0.3,0.2,16,0.002,100
3,3,5,0.245730,0.304824,0.122740,2.01,0.3,0.2,18,0.002,100


## 🔹 Lead V2 Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V2')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-04-55/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-18-23/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-31-50/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-44-27/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-56-57/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V2, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V2')
df_physionet_test_V2, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V2')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V2
     43/Unknown - 2s 39ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 39ms/step
     41/Unknown - 2s 36ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 36ms/step
     43/Unknown - 2s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     41/Unknown - 1s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     43/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V2
      9/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 28ms/step
      9/Unknown - 0s 25ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 26ms/step
    

In [ ]:
results_df_V2 = evaluate_models(df_physionet_train_V2,df_physionet_test_V2, hyperparams_list)

NameError: name 'df_physionet_train_V2' is not defined

In [ ]:
display(results_df_V2.sort_values(by="f1_macro", ascending=False))
#summarized_tables = summarize_by_latent_dimension(results_df_V2)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,3,0.225178,0.293479,0.136785,2.01,0.3,0.2,18,0.002,100
4,4,3,0.215569,0.291139,0.136037,2.01,0.3,0.2,22,0.002,100
1,1,3,0.208630,0.286224,0.134584,2.01,0.3,0.2,24,0.002,100
0,0,5,0.237811,0.302828,0.128966,6.01,0.3,0.2,20,0.002,100
2,2,3,0.212811,0.278506,0.128149,2.01,0.3,0.2,16,0.002,100


## 🔹 Lead V3 Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V2')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-04-55/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-18-23/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-31-50/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-44-27/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V2/2025-03-29_05-56-57/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V3, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V3')
df_physionet_test_V3, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V3')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V3
     43/Unknown - 4s 83ms/step✅ Generator exhausted normally.
44/44 [==============================] - 4s 82ms/step
     43/Unknown - 3s 72ms/step✅ Generator exhausted normally.
44/44 [==============================] - 3s 71ms/step
     43/Unknown - 2s 39ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 39ms/step
     42/Unknown - 1s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     42/Unknown - 2s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V3
     10/Unknown - 0s 27ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 30ms/step
      9/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 29ms/step
    

In [ ]:
results_df_V3 = evaluate_models(df_physionet_train_V3,df_physionet_test_V3, hyperparams_list)


🔍 Evaluating model 1/5...



🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_V3.sort_values(by="f1_macro", ascending=False))
#summarized_tables = summarize_by_latent_dimension(results_df_V2)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,3,0.242349,0.296655,0.133611,6.01,0.3,0.2,20,0.002,100
2,2,3,0.226601,0.290172,0.130353,2.01,0.3,0.2,16,0.002,100
3,3,3,0.260409,0.300849,0.129976,2.01,0.3,0.2,18,0.002,100
4,4,3,0.232918,0.298577,0.126163,2.01,0.3,0.2,22,0.002,100
1,1,3,0.255783,0.296323,0.125629,2.01,0.3,0.2,24,0.002,100


## 🔹 Lead V4 Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V4')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_03-34-32/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_03-47-08/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_03-59-27/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_04-11-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V4/2025-03-29_04-24-22/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V4, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V4')
df_physionet_test_V4, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V4')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V4
     43/Unknown - 2s 38ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 38ms/step
     43/Unknown - 2s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     42/Unknown - 1s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 34ms/step
     43/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     41/Unknown - 1s 34ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V4
      9/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 28ms/step
     10/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 29ms/step
    

In [ ]:
results_df_V4 = evaluate_models(df_physionet_train_V4,df_physionet_test_V4, hyperparams_list)


🔍 Evaluating model 1/5...



🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_V4.sort_values(by="f1_macro", ascending=False))
#summarized_tables = summarize_by_latent_dimension(results_df_V2)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,3,0.258185,0.304395,0.131029,6.01,0.3,0.2,20,0.002,100
2,2,3,0.226512,0.296367,0.130109,2.01,0.3,0.2,16,0.002,100
4,4,3,0.238790,0.295293,0.123494,2.01,0.3,0.2,22,0.002,100
3,3,5,0.258808,0.294518,0.118504,2.01,0.3,0.2,18,0.002,100
1,1,7,0.274555,0.301246,0.112992,2.01,0.3,0.2,24,0.002,100


## 🔹 Lead V5 Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V5')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_04-36-50/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_04-49-20/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_05-01-44/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_05-14-06/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V5/2025-03-29_05-26-31/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V5, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V5')
df_physionet_test_V5, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V5')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V5
     42/Unknown - 2s 56ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 54ms/step
     43/Unknown - 2s 37ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 37ms/step
     43/Unknown - 2s 39ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 39ms/step
     43/Unknown - 2s 36ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 36ms/step
     41/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V5
     10/Unknown - 0s 27ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
     10/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
    

In [ ]:
results_df_V5 = evaluate_models(df_physionet_train_V5,df_physionet_test_V5, hyperparams_list)


🔍 Evaluating model 1/5...

🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_V5.sort_values(by="f1_macro", ascending=False))
#summarized_tables = summarize_by_latent_dimension(results_df_V2)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,3,0.258363,0.314567,0.140270,2.01,0.3,0.2,16,0.002,100
1,1,3,0.273843,0.321651,0.140222,2.01,0.3,0.2,24,0.002,100
4,4,3,0.260943,0.313564,0.132063,2.01,0.3,0.2,22,0.002,100
0,0,3,0.237544,0.302725,0.130556,6.01,0.3,0.2,20,0.002,100
3,3,3,0.255427,0.310122,0.128155,2.01,0.3,0.2,18,0.002,100


## 🔹 Lead V6 Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = os.path.join(PATH,'V6')  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_05-38-58/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_05-51-18/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_06-03-46/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_06-16-04/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/full_set/test_is_split1/V6/2025-03-29_06-28-43/model_best.keras
Loaded 5 models successfully.


In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train_V6, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='V6')
df_physionet_test_V6, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='V6')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']
🔍 Processing lead: V6
     41/Unknown - 2s 37ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 37ms/step
     41/Unknown - 1s 33ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     43/Unknown - 1s 31ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step
     43/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 33ms/step
     43/Unknown - 1s 32ms/step✅ Generator exhausted normally.
44/44 [==============================] - 1s 32ms/step

📦 Loading dataset 'physionet' with splits: ['split1']
🔍 Processing lead: V6
     10/Unknown - 0s 26ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 27ms/step
      8/Unknown - 0s 28ms/step✅ Generator exhausted normally.
11/11 [==============================] - 0s 29ms/step
    

In [ ]:
results_df_V6 = evaluate_models(df_physionet_train_V6,df_physionet_test_V6, hyperparams_list)


🔍 Evaluating model 1/5...

🔍 Evaluating model 2/5...

🔍 Evaluating model 3/5...

🔍 Evaluating model 4/5...

🔍 Evaluating model 5/5...


In [ ]:
display(results_df_V6.sort_values(by="f1_macro", ascending=False))
summarized_tables = summarize_by_latent_dimension(results_df_V6)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,5,0.272420,0.317695,0.133943,2.01,0.3,0.2,22,0.002,100
3,3,5,0.271797,0.311583,0.130578,2.01,0.3,0.2,18,0.002,100
2,2,5,0.268772,0.315186,0.126885,2.01,0.3,0.2,16,0.002,100
1,1,3,0.228826,0.293171,0.122945,2.01,0.3,0.2,24,0.002,100
0,0,5,0.274377,0.314188,0.121019,6.01,0.3,0.2,20,0.002,100



📏 Latent Dimension: 16


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
2,2,5,0.268772,0.315186,0.126885,2.01,0.3,0.2,16,0.002,100



📏 Latent Dimension: 18


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
3,3,5,0.271797,0.311583,0.130578,2.01,0.3,0.2,18,0.002,100


...

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,5,0.274377,0.314188,0.121019,6.01,0.3,0.2,20,0.002,100



📏 Latent Dimension: 22


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,5,0.27242,0.317695,0.133943,2.01,0.3,0.2,22,0.002,100



📏 Latent Dimension: 24


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,3,0.228826,0.293171,0.122945,2.01,0.3,0.2,24,0.002,100


## 🔹 Special Evaluation


In [ ]:
import os
import tensorflow as tf

BASE = '/users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II'  # Set the correct path to your BASE directory
models = []
hyperparams_list = []

# Get all model folders in BASE directory
folders = [f for f in os.listdir(BASE) if os.path.isdir(os.path.join(BASE, f))]

for model in folders:
    model_path = os.path.join(BASE,model,'model_best.keras')
    hyperparams_path = os.path.join(BASE,model,'params.json')
    hyperparams = extract_hyperparams_from_json(hyperparams_path)
    hyperparams_list.append(hyperparams)
    if os.path.exists(model_path):
        print(f"Loading model from: {model_path}")
        model = tf.keras.models.load_model(model_path,compile=False)
        models.append(model)
    else:
        print(f"Warning: Model file not found at {model_path}")

print(f"Loaded {len(models)} models successfully.")

Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-14-42/model_best.keras


Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-22-20/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-30-02/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-37-56/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_13-45-54/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-28-26/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-36-43/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/latent_experiment/test_is_split1/II/2025-03-28_14-44-48/model_best.keras
Loading model from: /users/newc6477/VAE/12_Lead_VECG/results/l

In [ ]:
train_splits = ['split2', 'split3', 'split4', 'split5']
dataset_config = {
    'name': ['physionet'],
    'split': train_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
test_splits = ['split1']
dataset_test = {
    'name': ['physionet'],
    'split': test_splits,
    'shuffle_size': 1024,
    'batch_size': 1024,
}
df_physionet_train, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_config, lead='II')
df_physionet_test, ld = Helper.get_embeddings_multiple_model(models, datasets=dataset_test, lead='II')


📦 Loading dataset 'physionet' with splits: ['split2', 'split3', 'split4', 'split5']


🔍 Processing lead: II
     43/Unknown - 2s 50ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 50ms/step
     42/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 45ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 48ms/step
     43/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 46ms/step
     42/Unknown - 2s 47ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 47ms/step
     42/Unknown - 2s 46ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 46ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 47ms/step
     43/Unknown - 2s 48ms/step✅ Generator exhausted normally.
44/44 [==============================] - 2s 48ms/step
     43/Unknown - 2s 46ms/step✅ Generator exhauste

In [ ]:
results_df = evaluate_models(df_physionet_train,df_physionet_test, hyperparams_list)


🔍 Evaluating model 1/10...



🔍 Evaluating model 2/10...

🔍 Evaluating model 3/10...

🔍 Evaluating model 4/10...

🔍 Evaluating model 5/10...

🔍 Evaluating model 6/10...

🔍 Evaluating model 7/10...

🔍 Evaluating model 8/10...

🔍 Evaluating model 9/10...

🔍 Evaluating model 10/10...


In [ ]:
display(results_df.sort_values(by="f1_macro", ascending=False))
summarized_tables = summarize_by_latent_dimension(results_df)

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
4,4,3,0.270996,0.326328,0.146603,2.01,0.3,0.2,20,0.002,100
8,8,3,0.252580,0.314773,0.143990,2.01,0.3,0.2,28,0.002,100
6,6,3,0.259520,0.329740,0.143175,2.01,0.3,0.2,24,0.002,100
2,2,3,0.250890,0.319867,0.142723,2.01,0.3,0.2,16,0.002,100
1,1,3,0.258185,0.316899,0.138138,2.01,0.3,0.2,14,0.002,100
5,5,7,0.275445,0.329102,0.137830,2.01,0.3,0.2,22,0.002,100
7,7,3,0.244306,0.305650,0.136028,2.01,0.3,0.2,26,0.002,100
9,9,3,0.268505,0.318874,0.134784,2.01,0.3,0.2,30,0.002,100
0,0,3,0.241904,0.313710,0.134592,2.01,0.3,0.2,12,0.002,100
3,3,5,0.276601,0.328372,0.134307,2.01,0.3,0.2,18,0.002,100



📏 Latent Dimension: 12


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
0,0,3,0.241904,0.31371,0.134592,2.01,0.3,0.2,12,0.002,100



📏 Latent Dimension: 14


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
1,1,3,0.258185,0.316899,0.138138,2.01,0.3,0.2,14,0.002,100


...

,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
7,7,3,0.244306,0.30565,0.136028,2.01,0.3,0.2,26,0.002,100



📏 Latent Dimension: 28


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
8,8,3,0.25258,0.314773,0.14399,2.01,0.3,0.2,28,0.002,100



📏 Latent Dimension: 30


,model_index,best_k,accuracy,f1_micro,f1_macro,alpha,beta,gamma,latent_dimension,learning_rate,epochs
9,9,3,0.268505,0.318874,0.134784,2.01,0.3,0.2,30,0.002,100
